## Constituency Parsing

This notebook covers submodules of nltk.parse package.

<b>Parsing or syntax analysis or syntactic analysis</b> is the analysis of a string of symbols, conforming to the rules of a formal grammar.

#### Parsing using nltk.parse package

Reference - [nltk.parse package](https://www.nltk.org/api/nltk.parse.html)

In [ ]:
import nltk
from nltk import nonterminals, Production, CFG
from nltk.parse.generate import generate

Each node in the parse tree is a constituent. A grammar specifies how the sentence can be subdivided into its immediate constituents

A <b>context-free grammar</b> G=(V,T,P,S) is composed of

V : a set of variables / non-terminals

T : a set of terminal symbols 

P : a set of productions, rules that recursively define the structure of the language.

S : a starting symbol. 

A production has the form A→α where,
A is a variable.
α is a string of zero or more symbol, either terminals or variables.

In [ ]:
S, NP, VP = nonterminals('S, NP, VP')
N, V, P, DT = nonterminals('N, V, P, DT')

rule1 = Production(S, [NP, VP])
rule2 = Production(NP, [DT, NP])

print(rule1)
print(rule2)

S -> NP VP
NP -> DT NP


In [ ]:
rule1.rhs()

(NP, VP)

In [ ]:
grammar = CFG(S,[rule1,rule2])

grammar.productions()

[S -> NP VP, NP -> DT NP]

In [ ]:
grammar = CFG.fromstring("""
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> "saw" | "ate" | "walked" | "slept"
NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
Det -> "A" | "An" | "The" | "My" | "the" | "a"
N -> "man" | "dog" | "cat" | "telescope" | "park"
P -> "in" | "on" | "by" | "with"
""")
grammar.productions()

[S -> NP VP,
 VP -> V NP,
 VP -> V NP PP,
 PP -> P NP,
 V -> 'saw',
 V -> 'ate',
 V -> 'walked',
 NP -> 'John',
 NP -> 'Mary',
 NP -> 'Bob',
 NP -> Det N,
 NP -> Det N PP,
 Det -> 'A',
 Det -> 'An',
 Det -> 'The',
 Det -> 'My',
 Det -> 'the',
 Det -> 'a',
 N -> 'man',
 N -> 'dog',
 N -> 'cat',
 N -> 'telescope',
 N -> 'park',
 P -> 'in',
 P -> 'on',
 P -> 'by',
 P -> 'with']

In [ ]:
#generating sample sentences from the CFG
for sentence in generate(grammar, n=10):
  print(' '.join(sentence))

John saw John
John saw Mary
John saw Bob
John saw A man
John saw A dog
John saw A cat
John saw A telescope
John saw A park
John saw An man
John saw An dog


#### Recursive Descent Parser

<b>Recursive descent</b> is a top-down parsing technique that constructs the parse tree from the top and the input is read from left to right. 

The basic idea of recursive-descent parsing is to associate each non-terminal with a production. The goal of each such production is to read a sequence of input characters that can be generated by the corresponding non-terminal, and return a pointer to the root of the parse tree for the non-terminal. 

The terminal symbol is compared to the input; if they agree, it consumes the terminal symbol in the input or 'match'.
For a non-terminal symbol, the corresponding production is called or 'expand'.


In [ ]:
from nltk.parse import RecursiveDescentParser
rd = RecursiveDescentParser(grammar,trace=0)

In [ ]:
from nltk.parse.recursivedescent import demo
demo()

S -> NP VP
NP -> Det N
NP -> Det N PP
VP -> V NP
VP -> V NP PP
PP -> P NP
NP -> 'I'
N -> 'man'
N -> 'park'
N -> 'telescope'
N -> 'dog'
Det -> 'the'
Det -> 'a'
P -> 'in'
P -> 'with'
V -> 'saw'
Parsing 'I saw a man in the park'
    [ * S ]
  E [ * NP VP ]
  E [ * Det N VP ]
  E [ * 'the' N VP ]
  E [ * 'a' N VP ]
  E [ * Det N PP VP ]
  E [ * 'the' N PP VP ]
  E [ * 'a' N PP VP ]
  E [ * 'I' VP ]
  M [ 'I' * VP ]
  E [ 'I' * V NP ]
  E [ 'I' * 'saw' NP ]
  M [ 'I' 'saw' * NP ]
  E [ 'I' 'saw' * Det N ]
  E [ 'I' 'saw' * 'the' N ]
  E [ 'I' 'saw' * 'a' N ]
  M [ 'I' 'saw' 'a' * N ]
  E [ 'I' 'saw' 'a' * 'man' ]
  M [ 'I' 'saw' 'a' 'man' ]
  E [ 'I' 'saw' 'a' * 'park' ]
  E [ 'I' 'saw' 'a' * 'telescope' ]
  E [ 'I' 'saw' 'a' * 'dog' ]
  E [ 'I' 'saw' * Det N PP ]
  E [ 'I' 'saw' * 'the' N PP ]
  E [ 'I' 'saw' * 'a' N PP ]
  M [ 'I' 'saw' 'a' * N PP ]
  E [ 'I' 'saw' 'a' * 'man' PP ]
  M [ 'I' 'saw' 'a' 'man' * PP ]
  E [ 'I' 'saw' 'a' 'man' * P NP ]
  E [ 'I' 'saw' 'a' 'man' * 'in' NP ]
  

In [ ]:
sentence1 = 'Mary saw Bob'.split()
sentence2 = 'The dog saw a man in the park'.split()
print(sentence1,"\n",sentence2)

['Mary', 'saw', 'Bob'] 
 ['The', 'dog', 'saw', 'a', 'man', 'in', 'the', 'park']


In [ ]:
trees = rd.parse(sentence1)

for t in trees:
  print(t)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [ ]:
# the sentence does not belong to the grammar
for t in rd.parse("Mary Bob".split()):
  print(t)

In [ ]:
for t in rd.parse(sentence2):
  print(t)

(S
  (NP (Det The) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man) (PP (P in) (NP (Det the) (N park))))))
(S
  (NP (Det The) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man))
    (PP (P in) (NP (Det the) (N park)))))


##### Left recursion example

A left recursive production can send RD parser into an infinite loop.

In [ ]:
## NP -> NP P is the left recursive production
lrgrammar = CFG.fromstring("""
S -> NP V 
NP -> NP P | "John" | "Mary" | "Bob" | "Tom" 
P -> "are"
V -> "beautiful"
""")
lrgrammar.productions()

[S -> NP V,
 NP -> NP P,
 NP -> 'John',
 NP -> 'Mary',
 NP -> 'Bob',
 NP -> 'Tom',
 P -> 'are',
 V -> 'beautiful']

In [ ]:
lrparser = RecursiveDescentParser(lrgrammar)

In [ ]:
sent = "John Mary Bob Tom are beautiful".split()
for t in lrparser.parse(sent):
  print(t)

RecursionError: ignored

#### Shift Reduce Parser

Top-down parsers use a grammar to predict what the input will be, before inspecting the input. Since the input is available all along, it would be more sensible to consider the input sentence from the very beginning.

The advantage of shift-reduce parsers over recursive descent parsers is that they only build structure that corresponds to the words in the input.

*Shift Reduce parser* follows bottom-up parsing technique, constructing parse tree is constructed from leaves(bottom) to the root(up).

Shift : The parser repeatedly pushes the next input word onto a stack. 

Reduce : Replacement of the stack's top with a single non-terminal.

If the top n items on the stack match the n items on the right hand side of some production, then they are all popped off the stack, and the item on the left-hand side of the production is pushed on the stack. The parser finishes when all the input is consumed and there is only one item remaining on the stack, a parse tree with an S node as its root.

In [ ]:
from nltk.parse import ShiftReduceParser
sr = ShiftReduceParser(grammar,trace=0)

In [ ]:
from nltk.parse.shiftreduce import demo
demo()

Parsing 'I saw a man in the park'
    [ * I saw a man in the park]
  S [ 'I' * saw a man in the park]
  R [ NP * saw a man in the park]
  S [ NP 'saw' * a man in the park]
  R [ NP V * a man in the park]
  S [ NP V 'a' * man in the park]
  R [ NP V Det * man in the park]
  S [ NP V Det 'man' * in the park]
  R [ NP V Det N * in the park]
  R [ NP V NP * in the park]
  R [ NP VP * in the park]
  R [ S * in the park]
  S [ S 'in' * the park]
  R [ S P * the park]
  S [ S P 'the' * park]
  R [ S P Det * park]
  S [ S P Det 'park' * ]
  R [ S P Det N * ]
  R [ S P NP * ]
  R [ S PP * ]


In [ ]:
#sentence1 = 'Mary saw Bob'

for t in sr.parse(sentence1):
  print(t)

(S (NP Mary) (VP (V saw) (NP Bob)))


This parser does not implement any backtracking, so it is not guaranteed to find a parse for a text, even if one exists.

A shift-reduce parser can reach a dead end and fail to find any parse, even if the input sentence is well-formed according to the grammar. The problem arises because there are choices made earlier that cannot be undone by the parser.

In [ ]:
#sentence2 = 'The dog saw a man in the park'
tree = sr.parse(sentence2)
for t in tree:
  print(t)

In [ ]:
#trace for sentence2
sr = ShiftReduceParser(grammar,trace=2)
tree = sr.parse(sentence2)
for t in tree:
  print(t)

Parsing 'The dog saw a man in the park'
    [ * The dog saw a man in the park]
  S [ 'The' * dog saw a man in the park]
  R [ Det * dog saw a man in the park]
  S [ Det 'dog' * saw a man in the park]
  R [ Det N * saw a man in the park]
  R [ NP * saw a man in the park]
  S [ NP 'saw' * a man in the park]
  R [ NP V * a man in the park]
  S [ NP V 'a' * man in the park]
  R [ NP V Det * man in the park]
  S [ NP V Det 'man' * in the park]
  R [ NP V Det N * in the park]
  R [ NP V NP * in the park]
  R [ NP VP * in the park]
  R [ S * in the park]
  S [ S 'in' * the park]
  R [ S P * the park]
  S [ S P 'the' * park]
  R [ S P Det * park]
  S [ S P Det 'park' * ]
  R [ S P Det N * ]
  R [ S P NP * ]
  R [ S PP * ]


Resources :

[Analyzing sentence structure](https://www.nltk.org/book/ch08.html#code-cfg2)

[Parsing with corenlp](https://github.com/nltk/nltk/wiki/Stanford-CoreNLP-API-in-NLTK)

## Dependency Parsing

<b>Dependency grammar</b> focusses on how words relate to other words, instead of how words and sequences of words combine to form constituents.
 Dependency is a binary asymmetric relation that holds between a head and its dependents.

The head of a sentence is usually taken to be the tensed verb, and every other word is either dependent on the sentence head, or connects to it through a path of dependencies.

[Universal dependency relations](https://universaldependencies.org/u/dep/) contains list of possible dependency relations in the graph.

#### 1. Dependency parsing using spaCy

In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
doc = nlp("My brother’s dog barks a lot")
displacy.render(doc,style='dep',jupyter=True, options={'distance': 150})

In [3]:
doc = nlp("Cats hate water")
displacy.render(doc,style='dep',jupyter=True, options={'distance': 100})
for token in doc:
    print(token.text,"\t", token.dep_,"\t\t","Head of this token is :", token.head.text)

Cats 	 nsubj 		 Head of this token is : hate
hate 	 ROOT 		 Head of this token is : hate
water 	 dobj 		 Head of this token is : hate


In [4]:
doc = nlp("Children had fruits , snacks and cookies")
displacy.render(doc,style='dep',jupyter=True, options={'distance': 100})
for token in doc:
    print("{0}({1}-{2}, {3}-{4})".format(token.dep_, token.head.text, token.head.i+1, token.text,  token.i+1, ))

nsubj(had-2, Children-1)
ROOT(had-2, had-2)
dobj(had-2, fruits-3)
punct(fruits-3, ,-4)
conj(fruits-3, snacks-5)
cc(snacks-5, and-6)
conj(snacks-5, cookies-7)


In [5]:
token_array = [token for token in doc]
print("for the token 'Snacks', n_lefts = ",token_array[4].n_lefts," n_rights = ",token_array[4].n_rights)
print("for the token 'had', n_lefts = ",token_array[1].n_lefts," n_rights = ",token_array[1].n_rights)

for the token 'Snacks', n_lefts =  0  n_rights =  2
for the token 'had', n_lefts =  1  n_rights =  1


In [6]:
print("Is 'had' the ancestor of 'fruits'? ",token_array[1].is_ancestor(token_array[2]))


Is 'had' the ancestor of 'fruits'?  True


[Parsing English in 500 Lines of Python](https://explosion.ai/blog/parsing-english-in-python) is by MATTHEW HONNIBAL, who wrote spaCy.

#### 2. Dependency parsing using nltk

In [ ]:
from nltk.grammar import DependencyGrammar
from nltk.parse import DependencyGraph ,ProjectiveDependencyParser

In [ ]:
from nltk.parse.projectivedependencyparser import demo
demo()

Dependency grammar with 4 productions
  'scratch' -> 'cats'
  'scratch' -> 'walls'
  'walls' -> 'the'
  'cats' -> 'the'
(scratch (cats the) (walls the))
Training Probabilistic Projective Dependency Parser...
Parsing ' Cathy zag hen wild zwaaien . '...
Parse:
(zag Cathy hen (zwaaien wild .))


In [ ]:
dep_grammar = DependencyGrammar.fromstring("""
'shot' -> 'I' | 'elephant' | 'in'
'elephant' -> 'an' | 'in'
'in' -> 'pajamas'
'pajamas' -> 'my'
""")
print(dep_grammar)

Dependency grammar with 7 productions
  'shot' -> 'I'
  'shot' -> 'elephant'
  'shot' -> 'in'
  'elephant' -> 'an'
  'elephant' -> 'in'
  'in' -> 'pajamas'
  'pajamas' -> 'my'


In [ ]:
pdparser = ProjectiveDependencyParser(dep_grammar)
sent = 'I shot an elephant in Nigeria'.split()
trees = pdparser.parse(sent)
for t in trees:
     print(t)

In [ ]:
#visualise using displacy
doc = nlp("I shot an elephant in my pajamas")
displacy.render(doc,style='dep',jupyter=True, options={'distance': 100})

In [ ]:
grammar2 = DependencyGrammar.fromstring("""
'ate' -> 'children' | 'cake' | 'with' | 'spoon'
'children' -> 'The'
'cake' ->  'the'
'with' -> 'spoon'
'spoon' -> 'a'
""")
pdparser = ProjectiveDependencyParser(grammar2)


In [ ]:
sent = 'The children ate the cake with a spoon'.split()
trees = pdparser.parse(sent)
for t in trees:
     print(t)

(ate (children The) (cake the) with (spoon a))
(ate (children The) (cake the) (with (spoon a)))


In [ ]:
#visualise using displacy
doc = nlp("The city council refused permission to the protesters because they feared violence")
displacy.render(doc,style='dep',jupyter=True, options={'distance': 100})

##### Using Treebank data

In [ ]:
#from nltk.corpus import dependency_treebank

In [ ]:
# fields are word | tag | head | relation
treebank_data = """
Pierre  NNP     2       NMOD
Vinken  NNP     8       SUB
,       ,       2       P
61      CD      5       NMOD
years   NNS     6       AMOD
old     JJ      2       NMOD
,       ,       2       P
will    MD      0       ROOT
join    VB      8       VC
the     DT      11      NMOD
board   NN      9       OBJ
as      IN      9       VMOD
a       DT      15      NMOD
nonexecutive    JJ      15      NMOD
director        NN      12      PMOD
Nov.    NNP     9       VMOD
29      CD      16      NMOD
.       .       9       VMOD
"""

In [ ]:
dg = DependencyGraph(treebank_data)
dg.tree().pprint()

(will
  (Vinken Pierre , (old (years 61)) ,)
  (join (board the) (as (director a nonexecutive)) (Nov. 29) .))


## demo

In [ ]:
#!pip install -U nltk

In [ ]:
import nltk
from nltk.app import rdparser_app,srparser_app,chartparser_app

In [ ]:
rdparser_app.app()

In [ ]:
srparser_app.app()